In [17]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.polynomial.polynomial as poly

from helper import displayEpipolarF, calc_epi_error, toHomogenous, _singularize


In [2]:
    from scipy.spatial.transform import Rotation as sRot
    rotVec = sRot.random()
#     mat = rodrigues(rotVec.as_rotvec())


In [143]:
rotVec.reshape(-1, 1)

AttributeError: 'scipy.spatial.transform._rotation.Rotation' object has no attribute 'reshape'

In [7]:
u = rotVec.as_rotvec()

In [141]:
u.shape[0]

3

In [8]:
# u = r / theta
u1, u2, u3 = u[0, 0], u[1, 0], u[2, 0]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [125]:
    correspondence = np.load('data/some_corresp.npz') # Loading correspondences
    intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
    K1, K2 = intrinsics['K1'], intrinsics['K2']
    pts1, pts2 = correspondence['pts1'], correspondence['pts2']
    im1 = plt.imread('data/im1.png')
    im2 = plt.imread('data/im2.png')

    indices = np.array([18, 19, 24, 54, 56, 82, 84])

    M = np.max([*im1.shape, *im2.shape])

#     Farray = sevenpoint(pts1[indices, :]

In [126]:
pts1, pts2 = pts1[indices, :], pts2[indices, :]

In [127]:
Farray = []
# ----- TODO -----
# YOUR CODE HERE
N = pts1.shape[0]

# normalize pts
pts1, pts2 = pts1/float(M), pts2/float(M)
x1s = pts1[:, 0] #(N,)
y1s = pts1[:, 1]
x2s = pts2[:, 0]
y2s = pts2[:, 1]

# construct columns of A
c0 = x2s * x1s
c1 = x2s * y1s
c2 = x2s
c3 = y2s * x1s
c4 = y2s * y1s
c5 = y2s
c6 = x1s
c7 = y1s
c8 = np.ones((N,), dtype=np.float32)

A = np.stack((c0, c1, c2, c3, c4, c5, c6, c7, c8), axis=1)

# solve a raw f
U, singular_vals, Vt = np.linalg.svd(A)
# fk is the last kth column of V, so the last raw of Vt
f1, f2 = Vt[-1, :], Vt[-2, :] #(9,)
F1, F2 = f1.reshape(3, 3), f2.reshape(3, 3)

a, b = F1-F2, F2

fun = lambda x: np.linalg.det(x*a + b)
fun0 = fun(0)
fun1 = fun(1)
fun_1 = fun(-1)
fun2 = fun(2)
fun_2 = fun(-2)

c0 = fun0
c1 = (2.0/3)*(fun1-fun_1) - (1.0/12)*(fun2-fun_2)
c3 = (1.0/12)*(fun2 - fun_2) - (1.0/6)*(fun1-fun_1)
c2 = fun1 - c0 - c1 - c3

roots = np.roots([c3, c2, c1, c0])
roots_imag_mag = np.abs(np.imag(roots))
eps = 0.001
roots = roots[roots_imag_mag < eps]
roots = np.real(roots)


In [128]:
pts1

array([[0.24375  , 0.4765625],
       [0.740625 , 0.6      ],
       [0.2375   , 0.3515625],
       [0.3984375, 0.3171875],
       [0.7828125, 0.2375   ],
       [0.1921875, 0.196875 ],
       [0.7390625, 0.1609375]])

In [129]:
roots

array([2.11250024, 0.35624117, 0.09019566])

In [130]:
roots_test = poly.polyroots([c0, c1, c2, c3])
# roots_test = poly.polyroots([c3, c2, c1, c0])
roots_test

array([0.09019566, 0.35624117, 2.11250024])

In [ ]:

def sevenpoint(pts1, pts2, M):

    Farray = []
    # ----- TODO -----
    # YOUR CODE HERE
    N = pts1.shape[0]

    # normalize pts
    pts1, pts2 = pts1/float(M), pts2/float(M)
    x1s = pts1[:, 0] #(N,)
    y1s = pts1[:, 1]
    x2s = pts2[:, 0]
    y2s = pts2[:, 1]
    
    # construct columns of A
    c0 = x2s * x1s
    c1 = x2s * y1s
    c2 = x2s
    c3 = y2s * x1s
    c4 = y2s * y1s
    c5 = y2s
    c6 = x1s
    c7 = y1s
    c8 = np.ones((N,), dtype=np.float32)
    
    A = np.stack((c0, c1, c2, c3, c4, c5, c6, c7, c8), axis=1)
    
    # solve a raw f
    U, singular_vals, Vt = np.linalg.svd(A)
    # fk is the last kth column of V, so the last raw of Vt
    f1, f2 = Vt[-1, :], Vt[-2, :] #(9,)
    F1, F2 = f1.reshape(3, 3), f2.reshape(3, 3)

    a, b = F1-F2, F2

    fun = lambda x: np.linalg.det(x*a + b)
    fun0 = fun(0)
    fun1 = fun(1)
    fun_1 = fun(-1)
    fun2 = fun(2)
    fun_2 = fun(-2)

    c0 = fun0
    c1 = (2.0/3)*(fun1-fun_1) - (1.0/12)*(fun2-fun_2)
    c3 = (1.0/12)*(fun2 - fun_2) - (1.0/6)*(fun1-fun_1)
    c2 = fun1 - c0 - c1 - c3

    roots = np.roots([c3, c2, c1, c0])
    roots_imag_mag = np.abs(np.imag(roots))
    eps = 0.001
    roots = roots[roots_imag_mag < eps]
    roots = np.real(roots)

    # now get the F for unormalized coordinates
    # normalization transform
    T = np.zeros((3, 3), dtype=np.float32)
    T[0, 0] = 1.0 / M
    T[1, 1] = 1.0 / M
    T[2, 2] = 1.0

    Fs = list()
    for root in roots:
        F_norm = root*a + b
        F_unnorm = T.transpose() @ F_norm @ T
        Fs.append(F_unnorm)
    
    return Fs

In [131]:
M1 = np.zeros((3, 4), dtype=np.float32)
M1[[0, 1, 2], [0, 1, 2]] = 1
M1

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [132]:
M1 = np.zeros((3, 4), dtype=np.float32)
M1[0,0] = 1
M1[1,1] = 1
M1[2,2] = 1
M1

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [136]:
r = 50
x1 = 100 
x2 = 200 
y1 = 300
y2 = 400
ll = np.array([1, 0, -(x1-r)], dtype=np.float32).reshape(-1, 1)
lt = np.array([0, 1, -(y1-r)], dtype=np.float32).reshape(-1, 1)
lr = np.array([1, 0, -(x1+r)], dtype=np.float32).reshape(-1, 1)
lb = np.array([0, 1, -(y1+r)], dtype=np.float32).reshape(-1, 1)

boundary_lines = [ll, lt, lr, lb]
# intersection points
# intersect_points = [np.cross(l.reshape(-1), epipolarline_img2.reshape(-1)).reshape(-1, 1) for l in boundary_lines]
# select the end points of search line segment

In [139]:
lt

array([[   0.],
       [   1.],
       [-250.]], dtype=float32)

In [154]:
import os

loop = 5
data_path = os.path.join('data/q6/','time'+str(loop)+'.npz')
image1_path = os.path.join('data/q6/','cam1_time'+str(loop)+'.jpg')
image2_path = os.path.join('data/q6/','cam2_time'+str(loop)+'.jpg')
image3_path = os.path.join('data/q6/','cam3_time'+str(loop)+'.jpg')

im1 = plt.imread(image1_path)
im2 = plt.imread(image2_path)
im3 = plt.imread(image3_path)

data = np.load(data_path)
pts1 = data['pts1']
pts2 = data['pts2']
pts3 = data['pts3']


In [157]:
pts3

array([[ 917,  419,  800],
       [1036,  409,  600],
       [ 795,  423,  700],
       [ 910,  411,  400],
       [ 960,  374, 1000],
       [1054,  360, 1000],
       [ 778,  369,  800],
       [ 844,  366,  500],
       [ 877,  321,  700],
       [ 954,  310,  800],
       [ 800,  330,  600],
       [ 869,  312,  700]])